In [1]:
import requests
import json
import os
import pandas as pd

/var/folders/50/mpwd5ck16_l903wrv_g8dkm80000gn/T/ipykernel_37640/1937072657.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
rawData = pd.read_csv('./foodData.csv')
print(rawData)

        Meal       Food Item  Quantity Unit
0  Breakfast       Skim Milk       250   ml
1      Lunch  Chicken Breast       200   gm
2      Lunch     Cooked Rice       150   gm
3      Lunch        Broccoli       150   gm
4      Lunch         Spinach       100   gm
5      Lunch     Bell pepper       100   gm


In [3]:
params = {
    "api_key": "gTqiZtrpzrFxI3VaU7youPWodWg45qhgNqxvEdY0"
}

In [5]:
def checkIfNutrientNameIsUseful(nutrient_name):
    # Convert the nutrient name to lowercase for case-insensitive comparison
    nutrient_name_lower = nutrient_name.lower()

    # Define a list of common, useful nutrient names
    useful_nutrient_names = ["calories", "protein", "fat", "carbohydrates", "fiber", "sodium", "potassium", "vitamin a", "vitamin c", "vitamin d", "vitamin e", "vitamin b6", "vitamin b12", "folate", "iron", "calcium", "magnesium", "zinc", "energy"]

    # Check if the nutrient name is in the list of useful nutrient names
    if any(name.lower() in nutrient_name_lower for name in useful_nutrient_names):
        return True
    else:
        return False

In [6]:
# Query for each of these food items
for index, row in rawData.iterrows():
    food_item = row['Food Item']
    print(food_item)
    url = f"https://api.nal.usda.gov/fdc/v1/foods/search?dataType=Foundation&query={food_item}"
    response = requests.get(url, params=params)

    if response.status_code == 200:
        try:
            food_item_information = response.json()["foods"][0]
            print(food_item_information['description'])
            print(food_item_information['foodCategory'])
            important_nutrients = []
            for nutrient_information in food_item_information['foodNutrients']:
                nutrient_name = nutrient_information['nutrientName']
                if(checkIfNutrientNameIsUseful(nutrient_name)):
                    # print(json.dumps(nutrient_information, indent=4))
                    important_nutrients.append({
                        "nutrientName": nutrient_name,
                        "unitName": nutrient_information["unitName"],
                        "value": nutrient_information["value"]
                    })
            print(json.dumps(important_nutrients, indent=4))       
        except Exception as e:
            print(f"Error: {e}")
            print(response.json())
    else:
        print("Error: Unable to fetch data")
    

Skim Milk
Milk, nonfat, fluid, with added vitamin A and vitamin D (fat free or skim)
Dairy and Egg Products
[
    {
        "nutrientName": "Magnesium, Mg",
        "unitName": "MG",
        "value": 12.5
    },
    {
        "nutrientName": "Sodium, Na",
        "unitName": "MG",
        "value": 41.0
    },
    {
        "nutrientName": "Potassium, K",
        "unitName": "MG",
        "value": 167
    },
    {
        "nutrientName": "Zinc, Zn",
        "unitName": "MG",
        "value": 0.45
    },
    {
        "nutrientName": "Total lipid (fat)",
        "unitName": "G",
        "value": 0.08
    },
    {
        "nutrientName": "Protein",
        "unitName": "G",
        "value": 3.43
    },
    {
        "nutrientName": "Vitamin D (D2 + D3), International Units",
        "unitName": "IU",
        "value": 44.0
    },
    {
        "nutrientName": "Energy",
        "unitName": "KCAL",
        "value": 34.0
    },
    {
        "nutrientName": "Vitamin D3 (cholecalciferol)",
    